# MNIST example - recognize digits from given images. 
- Use basic tensorflow to perform classifications (use one-hot encoding)
- Measure the accuracy

In [194]:
import tensorflow as tf    
import numpy as np
from tensorflow.keras.datasets import mnist

## 1. Load MNIST dataset

In [172]:
np.random.seed(101)

(x_train_pre, y_train_pre), (x_test_pre, y_test_pre) = mnist.load_data()

train_examples = x_train_pre.shape[0]
test_examples = x_test_pre.shape[0]
x_train = x_train_pre.reshape(train_examples, 784)
x_test = x_test_pre.reshape(test_examples, 784)

m_train = x_train.shape[0]
m_test = x_test.shape[0]

## 1.1 Setup onehot encoding vectors
- Convert the train and test labels to onehot vectors

In [173]:
# Create one-hot encoding for predictions
def set_onehot_vector(y_onehot, y_array, length): 
    col = -1
    for row in range(length):
        col = (y_train_pre[row]-1)
        y_onehot[row][col]=1

In [174]:
y_train = np.zeros((m_train,10)).astype(int)
y_test = np.zeros((m_test,10)).astype(int)

set_onehot_vector(y_train, y_train_pre, m_train)
set_onehot_vector(y_test, y_test_pre, m_test)

In [175]:
print(" Shape of Training examples: {}".format(x_train.shape))
print(" Shape of Training labels: {}".format(y_train.shape))
print(" Shape of Test examples: {}".format(x_test.shape))
print(" Shape of Test labels: {}".format(y_test.shape))
print(" Test examples: {}".format(m_test))
print(" Train examples: {}".format(m_train))

 Shape of Training examples: (60000, 784)
 Shape of Training labels: (60000, 10)
 Shape of Test examples: (10000, 784)
 Shape of Test labels: (10000, 10)
 Test examples: 10000
 Train examples: 60000


## 2. Forward propagation
- y = wx + b



In [176]:
w_random = np.random.uniform(low=0, high=1, size=7840).reshape(784,10).astype('f')
b_random = np.random.uniform(low=0, high=1, size=10).astype('f')

x = tf.placeholder(tf.float32, name="x", shape=[None, 784])
w = tf.Variable( w_random , name="w", shape=[784,10])
b = tf.Variable( b_random , name="b", shape=[10])

y_pred = tf.matmul(x,w) + b

y_placeholder = tf.placeholder(tf.float32, shape=[None,10], name="y_ph")

## 3. Define cost function

In [177]:
cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits( labels = y_placeholder, logits = y_pred ) )

## 4. Optimizer

In [178]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01)
cost_optimizer = optimizer.minimize(cost)

## 5. Train the model
- Train the model in batches

In [233]:
batch_size = 200

def next_batch():
    next_batch_index = np.random.randint(0,m_test-1, batch_size)
    return (x_train[next_batch_index], y_train[next_batch_index])

In [234]:
init = tf.global_variables_initializer()

w_trained = 0
b_trained = 0

with tf.Session() as sess:
    
    sess.run(init)
    epochs = 10
    
    for i in range(6000):
        x_next, y_next = next_batch()
        sess.run(cost_optimizer, feed_dict={x:x_next, y_placeholder:y_next})
    
    w_trained = sess.run(w)
    b_trained = sess.run(b)    

## 6. Predictions

In [236]:
y_predict = np.dot(w_trained.T , x_train[50]) + b_trained
print(" Acrtual {} Prediction {}  ".format(y_train[50], y_predict))

 Acrtual [0 0 1 0 0 0 0 0 0 0] Prediction [ 8524.12   8316.089 12044.454  8429.348 10345.102  7470.961  9552.432
  9777.988  9084.388  9435.707]  


## 7. Accuraty

In [257]:
y_test_predict_pre = np.dot(x_test , w_trained) + b_trained
y_test_predict_pre.shape

(10000, 10)

In [258]:
y_test_predict_onehot = np.zeros((y_test_predict_pre.shape)).astype(int)
y_test_predict_onehot.shape

(10000, 10)

In [259]:
print(y_test_predict_onehot.shape[0])
for i in range(y_test_predict_onehot.shape[0]):
    y_test_predict_onehot[i] = (y_test_predict_pre[i] == np.amax(y_test_predict_pre[i])).astype(int)


10000


In [260]:
# print(y_test_predict_onehot[5])
index = 0
print(" Actual         {} ".format(y_test[index]))
print(" Prediction     {} ".format(y_test_predict_onehot[index]))
print(" Prediction Pre {} ".format(y_test_predict_pre[index]))

 Actual         [0 0 0 0 1 0 0 0 0 0] 
 Prediction     [0 0 0 0 0 0 1 0 0 0] 
 Prediction Pre [ 5226.0376  9274.5    12562.499   8500.62    7807.5864  3992.4453
 13660.406  10718.683  10426.621   9180.856 ] 


In [261]:
y_test_predict_onehot
y_test
count = 0
for item in range(y_test.shape[0]):
    if np.array_equal(y_test_predict_onehot[item], y_test[item]):
       count = count + 1

    
print(" Accuracy is : {} ".format(count / y_test.shape[0]))

 Accuracy is : 0.1011 
